In [19]:
import glob,os,sys,sncosmo
#from sntd import models,fitting,io
import sntd
from copy import copy
from astropy.table import vstack
from astropy.io import ascii

photometry='dolphot'

if photometry=='dolphot':
    files=glob.glob(os.path.join('data','pat_refsdal',photometry,'stringent','*.dat'))
elif photometry=='hstphot':
    files=glob.glob(os.path.join('data','pat_refsdal',photometry,'*.dat'))
else:
    raise RuntimeError("Photometry not recognized.")

In [20]:
bands=[]
lcs=sntd.curveDict(telescopename='Hubble',object='Refsdal_'+photometry)
images=['s1','s2','s3','s4']
for im in images:
    imTab=None
    for f in [x for x in files if im in x]:
        tab=ascii.read(f,comment='=')
        tab.remove_column('flag')
        tab.rename_column('mhjd','time')
        band=f[f.find('_'+photometry)-5:f.find('_'+photometry)]
        bands.append(band)
        tab['band']=band
        tab['zpsys']='ab'
        tab['zp']=25.0
        if imTab is None:
            imTab=copy(tab)
        else:
            imTab=vstack([imTab,tab])
    curve=sntd.table_factory(tab,telescopename='Hubble',object=f[5:7].upper())
    lcs.add_curve(curve)
print(lcs)
    

Telescope: Hubble
Object: Refsdal_dolphot
Number of bands: 1

Metadata:
info:

------------------
Image: S1:
Bands: set(['F160W'])
Date Range: 56621.99607->57750.90555
Number of points: 66
------------------
Image: S2:
Bands: set(['F160W'])
Date Range: 56630.85412->57759.79335
Number of points: 67
------------------
Image: S3:
Bands: set(['F160W'])
Date Range: 56574.27831->57703.18321
Number of points: 65
------------------
Image: S4:
Bands: set(['F160W'])
Date Range: 56611.48774->57740.39963
Number of points: 67
------------------
